In [1]:
import pandas as pd
import numpy as np

In [2]:
df1=pd.read_csv("matches23.csv")
df2=pd.read_csv("deliveries23.csv")

Cleaning part


In [3]:
df1.isnull().sum()

id                    0
season                0
city                 51
date                  0
match_type            0
team1                 0
team2                 0
toss_winner           0
toss_decision         0
result                0
winner                5
win_by_runs          19
win_by_wickets      268
target_runs           3
target_overs          3
player_of_match       5
venue                 0
super_over            0
method             1003
umpire1               0
umpire2               0
dtype: int64

In [4]:
#i will drop the row where the city,win_by_runs,target_runs,target_overs,player_of_match
df1=df1[(~df1["target_runs"].isnull())]
df1=df1[~(df1['city'].isnull())]
df1=df1[~(df1['win_by_runs'].isnull())]

In [5]:
#i will fill the null values of the "win by wickets"
df1['win_by_wickets'].fillna(df1['win_by_wickets'].mode().iloc[0],inplace=True)

In [6]:
df1.isnull().sum()

id                   0
season               0
city                 0
date                 0
match_type           0
team1                0
team2                0
toss_winner          0
toss_decision        0
result               0
winner               0
win_by_runs          0
win_by_wickets       0
target_runs          0
target_overs         0
player_of_match      0
venue                0
super_over           0
method             936
umpire1              0
umpire2              0
dtype: int64

In [7]:
#i will keep those data which have not applide "dl" method. i.e i will delete those row where method is d/l

df1['method'].value_counts()
df1=df1[~(df1['method'].notnull())]
df1['method'].fillna(0,inplace=True)

In [8]:
df1.isnull().sum()

id                 0
season             0
city               0
date               0
match_type         0
team1              0
team2              0
toss_winner        0
toss_decision      0
result             0
winner             0
win_by_runs        0
win_by_wickets     0
target_runs        0
target_overs       0
player_of_match    0
venue              0
super_over         0
method             0
umpire1            0
umpire2            0
dtype: int64

cleanig df2


In [9]:
df2.isnull().sum()

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batter                   0
bowler                   0
non_striker              0
batsman_runs             0
extra_runs               0
total_runs               0
extras_type         230758
is_wicket                0
player_dismissed    231750
dismissal_kind      231750
fielder             235154
dtype: int64

In [10]:
df2['player_dismissed'].fillna(0,inplace=True)

In [11]:
df2['player_dismissed'].value_counts()

player_dismissed
0                 231750
RG Sharma            210
V Kohli              195
S Dhawan             188
RV Uthappa           180
                   ...  
RR Bhatkal             1
CJ McKay               1
V Pratap Singh         1
Sunny Gupta            1
DG Nalkande            1
Name: count, Length: 603, dtype: int64

In [12]:
# convert all the text to 1 but keep the 0 as it is
df2['player_dismissed']=df2['player_dismissed'].apply(lambda x:x if x==0 else 1)

In [13]:
df2['player_dismissed'].value_counts()

player_dismissed
0    231750
1     12067
Name: count, dtype: int64

In [14]:
#i may not need the df2[['extras_type',"dismissal_kind","fielder"]] so i keep  it intact .

In [15]:
#lastly i drop many row from the df1 .as df1 and df2 are inter connectd. i will keep those df2['match_id'] which are in the
#df1['id']
df2=df2[df2['match_id'].isin(df1['id'])]

populating new columns from the old columns for our model

populating (total_runs)


In [16]:
df2.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [17]:
total_score_df=df2[['match_id','inning','total_runs']].groupby(["match_id","inning"]).sum()
total_score_df.reset_index(inplace=True)

In [18]:
#we need to find the 1st innings score so that we create a column total run for df1
gt=total_score_df[total_score_df['inning']==1]

In [19]:
match_df=pd.merge(df1,gt,left_on="id",right_on="match_id")

In [20]:
match_df.drop(['match_id','inning'],axis=1,inplace=True) #we donot want match_id,innings.

In [21]:
match_df  #our main data frame

,id,season,city,date,match_type,team1,team2,toss_winner,toss_decision,result,...,win_by_wickets,target_runs,target_overs,player_of_match,venue,super_over,method,umpire1,umpire2,total_runs
0,335982,2008,Bangalore,18-04-2008,League,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,runs,...,0.0,223.0,20.0,BB McCullum,M Chinnaswamy Stadium,N,0,Asad Rauf,RE Koertzen,222
1,335983,2008,Chandigarh,19-04-2008,League,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,runs,...,0.0,241.0,20.0,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",N,0,MR Benson,SL Shastri,240
2,335984,2008,Delhi,19-04-2008,League,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,wickets,...,9.0,130.0,20.0,MF Maharoof,Feroz Shah Kotla,N,0,Aleem Dar,GA Pratapkumar,129
3,335985,2008,Mumbai,20-04-2008,League,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,wickets,...,5.0,166.0,20.0,MV Boucher,Wankhede Stadium,N,0,SJ Davis,DJ Harper,165
4,335986,2008,Kolkata,20-04-2008,League,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,wickets,...,5.0,111.0,20.0,DJ Hussey,Eden Gardens,N,0,BF Bowden,K Hariharan,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,1359543,2023,Mumbai,21-05-2023,League,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,wickets,...,0.0,201.0,20.0,C Green,"Wankhede Stadium, Mumbai",N,0,KN Ananthapadmanabhan,RJ Tucker,200
932,1359544,2023,Bengaluru,21-05-2023,League,Royal Challengers Bangalore,Gujarat Titans,Gujarat Titans,field,wickets,...,0.0,198.0,20.0,Shubman Gill,"M Chinnaswamy Stadium, Bengaluru",N,0,Nitin Menon,VK Sharma,197
933,1370350,2023,Chennai,23-05-2023,Qualifier 1,Chennai Super Kings,Gujarat Titans,Gujarat Titans,field,runs,...,0.0,173.0,20.0,RD Gaikwad,"MA Chidambaram Stadium, Chepauk, Chennai",N,0,AK Chaudhary,CB Gaffaney,172
934,1370351,2023,Chennai,24-05-2023,Eliminator,Mumbai Indians,Lucknow Super Giants,Mumbai Indians,bat,runs,...,0.0,183.0,20.0,Akash Madhwal,"MA Chidambaram Stadium, Chepauk, Chennai",N,0,BNJ Oxenford,VK Sharma,182


removing the old teams which are not playing

In [22]:
match_df['team1'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans'], dtype=object)

In [23]:
ipl_team=['Royal Challengers Bangalore', 'Punjab Kings',
          'Mumbai Indians', 'Kolkata Knight Riders',
          'Rajasthan Royals','Chennai Super Kings','Sunrisers Hyderabad',
          'Delhi Capitals','Lucknow Super Giants', 'Gujarat Titans']


len(ipl_team)

10

In [24]:
#changing the old name to new name

match_df['team1']=match_df['team1'].str.replace("Kings XI Punjab",'Punjab Kings')
match_df['team2']=match_df['team2'].str.replace("Kings XI Punjab",'Punjab Kings')

match_df['team1']=match_df['team1'].str.replace("Delhi Daredevils",'Delhi Capitals')
match_df['team2']=match_df['team2'].str.replace("Delhi Daredevils",'Delhi Capitals')

match_df['team1']=match_df['team1'].str.replace("Deccan Chargers",'Sunrisers Hyderabad')
match_df['team2']=match_df['team2'].str.replace("Deccan Chargers",'Sunrisers Hyderabad')


In [25]:
#removing the old team name
match_df=match_df[match_df['team1'].isin(ipl_team)]
match_df=match_df[match_df['team2'].isin(ipl_team)]

In [26]:
# we need our requirement columns
match_df.columns
match_df=match_df[['id','city','winner','total_runs']]

again some cleaning the df2

In [27]:
len(match_df['id'].unique())

827

In [28]:
df2=df2[df2['match_id'].isin(match_df['id'])]  #removing those match_id from df2 which are not in match_df

In [29]:
len(df2['match_id'].unique())

827

In [30]:
df2.head(2)

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,0,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,0,NaN,NaN


In [31]:
#updating the old team name to new team name in df2 also
df2['batting_team']=df2['batting_team'].str.replace("Kings XI Punjab",'Punjab Kings')
df2['bowling_team']=df2['bowling_team'].str.replace("Kings XI Punjab",'Punjab Kings')

df2['batting_team']=df2['batting_team'].str.replace("Delhi Daredevils",'Delhi Capitals')
df2['bowling_team']=df2['bowling_team'].str.replace("Delhi Daredevils",'Delhi Capitals')

df2['batting_team']=df2['batting_team'].str.replace("Deccan Chargers",'Sunrisers Hyderabad')
df2['bowling_team']=df2['bowling_team'].str.replace("Deccan Chargers",'Sunrisers Hyderabad')

ball by ball recorde we have now below

In [32]:
#joining the match_df to the df2
delivery_df=pd.merge(match_df,df2,right_on='match_id',left_on='id')
delivery_df.drop(['id'],axis=1,inplace=True) #we have 2 id columns so i remove one of them
delivery_df=delivery_df[delivery_df['inning']==2]
delivery_df.head(5)

,city,winner,total_runs_x,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
124,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,AB Dinda,W Jaffer,1,0,1,NaN,0,0,NaN,NaN
125,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,AB Dinda,R Dravid,0,1,1,wides,0,0,NaN,NaN
126,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,AB Dinda,R Dravid,0,0,0,NaN,0,0,NaN,NaN
127,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,AB Dinda,R Dravid,1,0,1,NaN,0,0,NaN,NaN
128,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,AB Dinda,W Jaffer,1,0,1,NaN,0,0,NaN,NaN


In [33]:
# chenge city name Bengaluru  to Bangalore  and Navi Mumbai to Mumbai
delivery_df['city'].replace({'Bengaluru':'Bangalore'},inplace=True)
delivery_df['city'].replace({"Navi Mumbai":'Mumbai'},inplace=True)

calculation for new column (runs left)

In [34]:
delivery_df['current_socre']=delivery_df[['match_id','total_runs_y']].groupby('match_id').cumsum()

In [35]:
delivery_df

,city,winner,total_runs_x,match_id,inning,batting_team,bowling_team,over,ball,batter,...,non_striker,batsman_runs,extra_runs,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_socre
124,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,W Jaffer,1,0,1,NaN,0,0,NaN,NaN,1
125,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,R Dravid,0,1,1,wides,0,0,NaN,NaN,2
126,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,R Dravid,0,0,0,NaN,0,0,NaN,NaN,2
127,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,...,R Dravid,1,0,1,NaN,0,0,NaN,NaN,3
128,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,...,W Jaffer,1,0,1,NaN,0,0,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198464,Ahmedabad,Gujarat Titans,233,1370352,2,Mumbai Indians,Gujarat Titans,17,4,JP Behrendorff,...,K Kartikeya,1,0,1,NaN,0,0,NaN,NaN,166
198465,Ahmedabad,Gujarat Titans,233,1370352,2,Mumbai Indians,Gujarat Titans,17,5,K Kartikeya,...,JP Behrendorff,4,0,4,NaN,0,0,NaN,NaN,170
198466,Ahmedabad,Gujarat Titans,233,1370352,2,Mumbai Indians,Gujarat Titans,17,6,K Kartikeya,...,JP Behrendorff,0,0,0,NaN,0,0,NaN,NaN,170
198467,Ahmedabad,Gujarat Titans,233,1370352,2,Mumbai Indians,Gujarat Titans,18,1,JP Behrendorff,...,K Kartikeya,1,0,1,NaN,0,0,NaN,NaN,171


In [36]:
delivery_df['runs_left']=(delivery_df['total_runs_x']-delivery_df['current_socre'])+1 #why +1 because if team 1 score 100 team 2 must score 101 to win
#thats why i add +1

In [37]:
delivery_df

,city,winner,total_runs_x,match_id,inning,batting_team,bowling_team,over,ball,batter,...,batsman_runs,extra_runs,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_socre,runs_left
124,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,1,0,1,NaN,0,0,NaN,NaN,1,222
125,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,0,1,1,wides,0,0,NaN,NaN,2,221
126,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,0,0,0,NaN,0,0,NaN,NaN,2,221
127,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,...,1,0,1,NaN,0,0,NaN,NaN,3,220
128,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,...,1,0,1,NaN,0,0,NaN,NaN,4,219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198464,Ahmedabad,Gujarat Titans,233,1370352,2,Mumbai Indians,Gujarat Titans,17,4,JP Behrendorff,...,1,0,1,NaN,0,0,NaN,NaN,166,68
198465,Ahmedabad,Gujarat Titans,233,1370352,2,Mumbai Indians,Gujarat Titans,17,5,K Kartikeya,...,4,0,4,NaN,0,0,NaN,NaN,170,64
198466,Ahmedabad,Gujarat Titans,233,1370352,2,Mumbai Indians,Gujarat Titans,17,6,K Kartikeya,...,0,0,0,NaN,0,0,NaN,NaN,170,64
198467,Ahmedabad,Gujarat Titans,233,1370352,2,Mumbai Indians,Gujarat Titans,18,1,JP Behrendorff,...,1,0,1,NaN,0,0,NaN,NaN,171,63


calculation for (balls_left)

In [38]:
# to working the formula 126-(delivery_df['over']*6+delivery_df['ball']) i need the over like 1-10 not in 0-9
a=[n for n in range(0,20)]
b=[n for n in range (1,21)]
d={} #{0:1,1:2}
for n in a:
  d[n]=b[a.index(n)]
d

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20}

In [39]:
delivery_df['over'].replace(d,inplace=True)

In [40]:
#now applying the formula
delivery_df['balls_left']=126 - (delivery_df['over']*6 + delivery_df['ball'])

(wicket) after each ball column.

In [41]:
delivery_df.rename(columns={'wicket':'wickets'},inplace=True)

In [42]:
delivery_df.head(1)

,city,winner,total_runs_x,match_id,inning,batting_team,bowling_team,over,ball,batter,...,extra_runs,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_socre,runs_left,balls_left
124,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,R Dravid,...,0,1,NaN,0,0,NaN,NaN,1,222,119


In [43]:
delivery_df['wickets']=delivery_df[['match_id','player_dismissed']].groupby('match_id').cumsum()

In [44]:
delivery_df['wickets']=10-delivery_df['wickets']

In [45]:
delivery_df.head(1)

,city,winner,total_runs_x,match_id,inning,batting_team,bowling_team,over,ball,batter,...,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_socre,runs_left,balls_left,wickets
124,Bangalore,Kolkata Knight Riders,222,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,R Dravid,...,1,NaN,0,0,NaN,NaN,1,222,119,10


calculation for crr(current runrate)

In [46]:
#crr= runs/over
delivery_df['crr']=(delivery_df['current_socre']*6)/(120-delivery_df['balls_left'])

calculation for rrr(required run rate)

In [47]:
delivery_df['rrr']=delivery_df['runs_left']*6/delivery_df['balls_left']

cleaning crr and rrr columns

In [48]:
delivery_df[['crr','rrr']].describe()

c:\Python311\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,crr,rrr
count,96226.000000,9.621200e+04
mean,7.487117,NaN
std,2.279710,NaN
min,0.000000,-inf
25%,6.318584,7.309091e+00
50%,7.542857,9.034483e+00
75%,8.740741,1.119231e+01
max,36.000000,inf


In [49]:
#tackeling the null values in rrr
delivery_df['rrr'].fillna(0,inplace=True)

In [50]:
#tackeling the inf value in rrr
delivery_df.loc[delivery_df['rrr']==np.inf,['rrr']]=500 #i take 500 randomly as to handel the inf value

In [51]:
#tackeling the -inf values in rrr
delivery_df.loc[delivery_df['rrr']==-(np.inf),['rrr']]=0.0

In [52]:
#lastly i round the decimal points of the crr and rrr to 2 decimal place
delivery_df['crr']=delivery_df['crr'].round(2)
delivery_df['rrr']=delivery_df['rrr'].round(2)

(result) columns


In [53]:
#if the batting team and the winner is same then i want 1. else 0

delivery_df['result']=delivery_df.apply(lambda x: 1 if x['winner']==x['batting_team'] else 0,axis=1)

**final data frame**

In [54]:
final_df=delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [55]:
final_df.isnull().sum()

batting_team    0
bowling_team    0
city            0
runs_left       0
balls_left      0
wickets         0
total_runs_x    0
crr             0
rrr             0
result          0
dtype: int64

In [56]:
#shuffling the data for non biased model creation
final_df=final_df.sample(final_df.shape[0])
final_df.reset_index(inplace=True)
final_df.drop('index',axis=1,inplace=True)

In [57]:
final_df.isnull().sum()

batting_team    0
bowling_team    0
city            0
runs_left       0
balls_left      0
wickets         0
total_runs_x    0
crr             0
rrr             0
result          0
dtype: int64

i have to convert all the string value to numeric for my model
(below i did it by the help of pipe line)

In [58]:
final_df.head(2)

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr,result
0,Royal Challengers Bangalore,Punjab Kings,Chandigarh,89,55,7,163,6.92,9.71,1
1,Chennai Super Kings,Delhi Capitals,Delhi,44,55,7,147,9.60,4.80,1


x,y separation

In [59]:
x=final_df.iloc[::,0:9]
y=final_df.iloc[::,9]

In [60]:
x.head(1)

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_runs_x,crr,rrr
0,Royal Challengers Bangalore,Punjab Kings,Chandigarh,89,55,7,163,6.92,9.71


In [61]:
y.head(1)

0    1
Name: result, dtype: int64

Model creation

In [62]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=1)

In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse_output=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [65]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])

In [66]:
pipe.fit(x_train,y_train)

c:\Python311\Lib\site-packages\sklearn\compose\_column_transformer.py:1624: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [67]:
y_pred=pipe.predict(x_test)

In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8177496038034865

In [69]:
# pipe.predict_proba(x_test)[10]

city and team name for our app

In [70]:
final_df['batting_team'].unique()

array(['Royal Challengers Bangalore', 'Chennai Super Kings',
       'Kolkata Knight Riders', 'Sunrisers Hyderabad', 'Delhi Capitals',
       'Lucknow Super Giants', 'Rajasthan Royals', 'Mumbai Indians',
       'Punjab Kings', 'Gujarat Titans'], dtype=object)

In [71]:
final_df['city'].unique()

array(['Chandigarh', 'Delhi', 'Mumbai', 'Raipur', 'Lucknow', 'Bangalore',
       'Kolkata', 'Hyderabad', 'Port Elizabeth', 'Pune', 'Jaipur',
       'East London', 'Ahmedabad', 'Durban', 'Sharjah', 'Dubai', 'Indore',
       'Chennai', 'Abu Dhabi', 'Dharamsala', 'Centurion', 'Visakhapatnam',
       'Cuttack', 'Ranchi', 'Kimberley', 'Bloemfontein', 'Guwahati',
       'Nagpur', 'Johannesburg', 'Cape Town'], dtype=object)

In [73]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))